# Moderation Analysis: Individual Differences in Response to AI Uncertainty

## Research Question

While our main hypothesis testing showed no significant main effect of uncertainty communication on trust in AI systems, **moderation analysis** asks a more nuanced question:

> **For whom or under what conditions does uncertainty communication affect trust?**

Even when an overall effect is null, different subgroups may respond differently to the intervention:
- High-tech-affinity individuals might trust the AI *more* when shown uncertainty (transparency valued)
- Low-tech-affinity individuals might trust it *less* (uncertainty perceived as incompetence)
- These opposing effects could cancel out, producing the observed null main effect

## Statistical Framework

### Moderation Analysis

We test moderation using multiple regression with interaction terms:

$$Y = \beta_0 + \beta_1 X + \beta_2 M + \beta_3 (X \times M) + \epsilon$$

Where:
- $Y$ = outcome (TiA subscale)
- $X$ = treatment (uncertainty vs control)
- $M$ = moderator (e.g., age, ATI)
- $\beta_3$ = **moderation effect** (interaction)
- $\beta_2$ = **direct effect** of moderator

### What We Extract

For each moderator-outcome pair, we extract TWO effects:

1. **Moderation Effect (β₃)**: Does the moderator change how uncertainty affects trust?
2. **Direct Effect (β₂)**: Does the moderator predict trust, regardless of experimental condition?

## Analysis Approach

- **Unified Analysis**: All moderators (theoretical + exploratory) analyzed together
- **Hypothesis Type Flagged**: Results marked as "theoretical" or "exploratory"
- **Interpretations Included**: Human-readable interpretations saved to CSV
- **Multiple Outputs**: Separate CSVs for moderation effects, direct effects, and correlation matrix

## Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests
from scripts.stats import (test_moderation, interpret_moderation, 
                           interpret_direct_effect, format_effect_with_stars)
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# define variable names
tia_scales = ['tia_f', 'tia_pro', 'tia_rc', 'tia_t', 'tia_up']
hcsds_scales = ['hcsds_c', 'hcsds_v']
ati_scales = ['ati']
manip_check_scales = []
for i in range(4):
    manip_check_scales.append(f'manip_check1_{i+1}')

scale_titles = {
    'ati': 'Affinity for Technology Interaction',
    'hcsds_c': 'Healthcare Trust - Competence',
    'hcsds_v': 'Healthcare Trust - Values',
    'tia_rc': 'TiA - Reliability/Confidence',
    'tia_up': 'TiA - Understanding/Predictability',
    'tia_f': 'TiA - Familiarity',
    'tia_pro': 'TiA - Propensity to Trust',
    'tia_t': 'TiA - Trust in Automation'
}

In [2]:
# Load data
data = pd.read_csv('../data/data_scales.csv')
print(f"Total sample size: {len(data)}")
print(f"\nGroup distribution:")
print(data['stimulus_group'].value_counts())

Total sample size: 255

Group distribution:
stimulus_group
1    129
0    126
Name: count, dtype: int64


## Data Preparation

### Variable Centering and Coding

We prepare variables for moderation analysis:

1. **Effect code treatment**: stimulus_group as -0.5 (control) and 0.5 (uncertainty)
2. **Standardize continuous variables**: For better comparison of beta values between variables
3. **Effect code categorical variables**: For symmetric interpretation

In [23]:
# Create working copy
df = data.copy()

# 1. Effect code treatment: control = -0.5, uncertainty = 0.5
df['group_effect'] = df['stimulus_group'] - 0.5

# 2. Normalize all continuous variables
continuous_vars = hcsds_scales + ati_scales + tia_scales + ['age', 'page_submit']

for var in continuous_vars:
    df[f'{var}_c'] = (df[var] - df[var].mean())/df[var].std()

# 3. Effect code gender: female = -0.5, "other/prefer not to say" = 0, male = 0.5
# TODO: implement new gender mapping
gender_mapping = df['gender'].value_counts().index[:2]  # Two most common
df_full = df.copy()  # Keep full dataset
df = df[df['gender'].isin(gender_mapping)].copy()  # Filter for gender analysis
df['gender_c'] = df['gender'].map({gender_mapping[0]: -0.5, gender_mapping[1]: 0.5})

# 4. Mean-center ordinal variables (education, AI experience)
df['education_c'] = df['education'] - df['education'].mean()
df['Q19_c'] = df['Q19'] - df['Q19'].mean()

print(f"Prepared {len(df)} observations for analysis")
print(f"Continuous moderators: {len(continuous_vars)}")
print(f"Total moderators to test: {len(continuous_vars) + 3}")  # + gender, education, Q19

Prepared 250 observations for analysis
Continuous moderators: 10
Total moderators to test: 13


## Define All Moderators/Predictors

Choose which variables to analyze as both moderators (moderating effect on group -> tia) and predictors (direct effect predictor -> tia)

In [26]:
# Define all moderators with their properties
moderators = hcsds_scales + ati_scales + ['age_c', 'gender_c', 'education_c', 'Q19_c', 'page_submit']

print(f"Total moderators: {len(moderators)}")
print(f"Total tests: {len(moderators) * len(tia_scales)} (moderators × outcomes)")

Total moderators: 8
Total tests: 40 (moderators × outcomes)


## Unified Moderation Analysis

We analyze all moderators in a single loop, extracting both moderation effects and direct effects simultaneously.

In [27]:
# Storage for results
moderation_results = []
direct_effect_results = []

independent = 'group_effect'

print("="*80)
print("RUNNING ANALYSIS")
print("="*80)
print(f"Testing {len(moderators)} moderators × {len(tia_scales)} outcomes = {len(moderators) * len(tia_scales)} tests\n")

# Main analysis loop
for mod in moderators:
    if mod in scale_titles:
        mod_name = scale_titles[mod]
    else:
        mod_name = mod
    
    print(f"Testing moderator: {mod_name}")
    
    for dependent in tia_scales:
        # Run moderation analysis
        results, model = test_moderation(df, independent, dependent, mod,
                                         name=mod_name)
        
        # Extract moderation effect (interaction)
        moderation_results.append({
            'moderator': mod_name,
            'dependent': dependent,
            'independent': independent,
            'beta': results['b_interaction'],
            'se': results['se_interaction'],
            'ci_lower': results['ci_interaction_lower'],
            'ci_upper': results['ci_interaction_upper'],
            'p': results['p_interaction'],
            'r_squared': results['r_squared'],
            'adj_r_squared': results['adj_r_squared'],
            'n': results['n'],
            'sig': '***' if results['p_interaction'] < 0.001 else '**' if results['p_interaction'] < 0.01 else '*' if results['p_interaction'] < 0.05 else 'ns',
            'interpretation': interpret_moderation(results['b_interaction'], results['p_interaction'], mod_name)
        })
        
        # Extract direct effect (main effect of moderator)
        direct_effect_results.append({
            'predictor': mod_name,
            'dependent': dependent,
            'beta': results['b_moderator'],
            'se': results['se_moderator'],
            'p': results['p_moderator'],
            'r_squared': results['r_squared'],
            'adj_r_squared': results['adj_r_squared'],
            'n': results['n'],
            'sig': '***' if results['p_moderator'] < 0.001 else '**' if results['p_moderator'] < 0.01 else '*' if results['p_moderator'] < 0.05 else 'ns',
            'interpretation': interpret_direct_effect(results['b_moderator'], results['p_moderator'], mod_name, scale_titles[dependent])
        })

print(f"\nCompleted {len(moderation_results)} tests")
print(f"Extracted {len(moderation_results)} moderation effects")
print(f"Extracted {len(direct_effect_results)} direct effects")

RUNNING ANALYSIS
Testing 8 moderators × 5 outcomes = 40 tests

Testing moderator: Healthcare Trust - Competence
Testing moderator: Healthcare Trust - Values
Testing moderator: Affinity for Technology Interaction
Testing moderator: age_c
Testing moderator: gender_c
Testing moderator: education_c
Testing moderator: Q19_c
Testing moderator: page_submit

Completed 40 tests
Extracted 40 moderation effects
Extracted 40 direct effects


## Results Summary

### Significant Moderation Effects

In [30]:
# Convert to DataFrames
mod_df = pd.DataFrame(moderation_results)
dir_df = pd.DataFrame(direct_effect_results)

# Filter significant moderation effects
sig_mod = mod_df[mod_df['p'] < 0.05].copy()

print("="*80)
print(f"SIGNIFICANT MODERATION EFFECTS (p < .05): {len(sig_mod)}/{len(mod_df)} ({len(sig_mod)/len(mod_df)*100:.1f}%)")
print("="*80)

if len(sig_mod) > 0:
    sig_mod_display = sig_mod[['moderator', 'dependent', 'beta', 'r_squared', 'p', 'sig', 'interpretation']].copy()
    print(sig_mod_display.to_string(index=False))
else:
    print("No significant moderation effects found.")

SIGNIFICANT MODERATION EFFECTS (p < .05): 2/40 (5.0%)
  moderator dependent   beta  r_squared      p sig                                                                                        interpretation
education_c     tia_t 0.1773     0.0536 0.0104   * Significant moderation (p = 0.010): The effect of "intervention" increases as "education_c" increases
page_submit     tia_t 0.0025     0.0257 0.0470   * Significant moderation (p = 0.047): The effect of "intervention" increases as "page_submit" increases


### Significant Direct Effects

In [31]:
# Filter significant direct effects
sig_dir = dir_df[dir_df['p'] < 0.05].copy()

print("="*80)
print(f"SIGNIFICANT DIRECT EFFECTS (p < .05): {len(sig_dir)}/{len(dir_df)} ({len(sig_dir)/len(dir_df)*100:.1f}%)")
print("="*80)

if len(sig_dir) > 0:
    # Group by outcome
    for dependent in tia_scales:
        outcome_effects = sig_dir[sig_dir['dependent'] == dependent]
        if len(outcome_effects) > 0:
            print(f"\n{dependent} ({scale_titles[dependent]}):")
            for _, row in outcome_effects.iterrows():
                print(f"  • {row['predictor']}: β = {row['beta']:.3f}, r2 = {row['r_squared']*100:.2f}%, p = {row['p']:.4f} {row['sig']}")
                print(f"    {row['interpretation']}")
else:
    print("No significant direct effects found.")

SIGNIFICANT DIRECT EFFECTS (p < .05): 16/40 (40.0%)

tia_f (TiA - Familiarity):
  • Affinity for Technology Interaction: β = 0.305, r2 = 7.84%, p = 0.0000 ***
    Significant positive effect (p < .001): Higher "Affinity for Technology Interaction" predicts higher "TiA - Familiarity"
  • age_c: β = 0.146, r2 = 3.32%, p = 0.0116 *
    Significant positive effect (p = 0.012): Higher "age_c" predicts higher "TiA - Familiarity"

tia_pro (TiA - Propensity to Trust):
  • Healthcare Trust - Competence: β = 0.122, r2 = 1.85%, p = 0.0450 *
    Significant positive effect (p = 0.045): Higher "Healthcare Trust - Competence" predicts higher "TiA - Propensity to Trust"
  • Healthcare Trust - Values: β = 0.194, r2 = 4.15%, p = 0.0014 **
    Significant positive effect (p = 0.001): Higher "Healthcare Trust - Values" predicts higher "TiA - Propensity to Trust"
  • age_c: β = 0.120, r2 = 3.41%, p = 0.0036 **
    Significant positive effect (p = 0.004): Higher "age_c" predicts higher "TiA - Propensity to

## Effects Matrix

Matrix showing all predictor→outcome relationships with APA-style significance markers.

In [15]:
# Create effects matrix (predictors × outcomes)
matrix_data = {}
for outcome in tia_scales:
    matrix_data[outcome] = {}
    for predictor in dir_df['predictor'].unique():
        row = dir_df[(dir_df['predictor'] == predictor) & (dir_df['dependent'] == outcome)].iloc[0]
        matrix_data[outcome][predictor] = format_effect_with_stars(row['beta'], row['p'])

effects_matrix = pd.DataFrame(matrix_data).T

print("\n" + "="*100)
print("EFFECTS MATRIX: Predictor → Outcome Relationships")
print("="*100)
print("\nRows = Predictors | Columns = TiA Outcomes")
print("Cell values = β coefficient with significance: * p<.05, ** p<.01, *** p<.001\n")
print(effects_matrix.to_string())
print(f"\n{'='*100}\n")


EFFECTS MATRIX: Predictor → Outcome Relationships

Rows = Predictors | Columns = TiA Outcomes
Cell values = β coefficient with significance: * p<.05, ** p<.01, *** p<.001

        Healthcare Trust - Competence Healthcare Trust - Values Affinity for Technology Interaction     age_c gender_c education_c   Q19_c
tia_f                          -0.071                    -0.033                            0.305***    0.011*    0.164       0.053   0.031
tia_pro                        0.122*                   0.194**                              -0.030   0.009**    0.128      -0.036   0.014
tia_rc                       0.203***                  0.172***                               0.019  0.011***    0.135       0.027  -0.000
tia_t                        0.278***                  0.246***                               0.039  0.014***   0.256*      0.079*   0.083
tia_up                         -0.034                    -0.012                             -0.138*     0.003    0.004       0.047  

## Export Results to CSV

Saving three CSV files:
1. **moderation_effects.csv**: All interaction effects with interpretations
2. **direct_effects.csv**: All predictor→outcome effects with interpretations
3. **effects_matrix.csv**: Matrix of coefficients with significance markers

In [17]:
# Save moderation effects
mod_df.to_csv('../output/moderation_effects.csv', index=False)
print("✓ Saved: output/moderation_effects.csv")
print(f"  - Total effects: {len(mod_df)}")
print(f"  - Significant (p < .05): {(mod_df['p'] < 0.05).sum()}")

# Save direct effects
dir_df.to_csv('../output/direct_effects.csv', index=False)
print("\n✓ Saved: output/direct_effects.csv")
print(f"  - Total effects: {len(dir_df)}")
print(f"  - Significant (p < .05): {(dir_df['p'] < 0.05).sum()}")

# Save effects matrix
effects_matrix.to_csv('../output/effects_matrix.csv')
print("\n✓ Saved: output/effects_matrix.csv")
print(f"  - Dimensions: {effects_matrix.shape[0]} predictors × {effects_matrix.shape[1]} outcomes")

✓ Saved: output/moderation_effects.csv
  - Total effects: 35
  - Significant (p < .05): 1

✓ Saved: output/direct_effects.csv
  - Total effects: 35
  - Significant (p < .05): 14

✓ Saved: output/effects_matrix.csv
  - Dimensions: 5 predictors × 7 outcomes
